<a href="https://colab.research.google.com/github/mykolesiko/eeg_investigation/blob/diplom/MADE_PSD_leak_beta1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Статья CONVOLUTIONAL NEURAL NETWORK APPROACH FOR EEG-BASED EMOTION
RECOGNITION USING BRAIN CONNECTIVITY AND ITS SPATIAL INFORMATION
https://drive.google.com/file/d/1KU_5ovXNHyrt15iZnm7tmZMoxxGltN_J/view?usp=sharing

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchtext
import random
import math
import time
import torch.nn.functional as F

from tqdm.notebook import tqdm
from torch import optim
from torch.utils.data import DataLoader, Dataset, WeightedRandomSampler
import numpy as np
from  scipy import stats
import scipy
np.random.seed(1)
torch.manual_seed(1)
random.seed(1)

In [ ]:
import os
os.chdir("/content/drive/MyDrive/MADE/Project/deap")

In [ ]:
class Args:
  def __init__(self): #(data_path, epoch, batch_siz, image_size, learning_rate, weight_deca, learning_rate, learning_rate_gamma, weight_bce, load, output_dir)
    self.data_path = "/content/drive/MyDrive/MADE/semester2/CV/contest02/data/"
    self.epochs = 2
    self.batch_size = 256
    self.lr= 3e-4
    self.weight_decay= 1e-6
    self.learning_rate=None
    self.learning_rate_gamma=None
    self.weight_bce=1
    self.load=None
    self.output_dir="/content/drive/MyDrive/MADE/Project/RACNN_models/"
    self.data_dir ="./data_preprocessed_python/"# "/content/drive/MyDrive/MADE/Project/train/physionet.org/"
args = Args()   

In [ ]:
import glob
import pickle
from collections import Counter
import scipy
from  scipy import signal
from scipy.fft import fft, fftfreq

In [ ]:
import glob
import pickle
from sklearn.preprocessing import MinMaxScaler

data = []
labels = []
data_dir = './data_preprocessed_python'
files = glob.glob(os.path.join(data_dir, "*.dat"))
data_raw = []
for file_data in files:
    raw_data = pickle.load(open(file_data, 'rb'), encoding='latin1')
    data.append(raw_data['data'])
    labels.append(raw_data['labels'])
    #break

In [ ]:
labels_bin = []
for sub in range(32):
#for sub in range(1):
  temp = labels[sub] >= 4.5
  labels_bin.append(temp)
  #print(sum(labels_bin[sub][:, type_emotion]), end=' ')

In [ ]:
print(len(data))

32


In [ ]:
LEN_RECORD_IN_SECONDS = 60
NVIDEOS = 40
HCANALS = 9
WCANALS = 9
NTIMES_IN_SAMPLE = 128
NTIMES_IN_SEC = 128
NCANALS = 32
NFEATURES = 32
NSUBJECTS = 32
electrode_matrix = {}
electrode_matrix['FP1'] = [0, 3]
electrode_matrix['FP2'] = [0, 5]
electrode_matrix['AF3'] = [1, 3]
electrode_matrix['AF4'] = [1, 5]
electrode_matrix['F7']  = [2, 0]
electrode_matrix['F3']  = [2, 2]
electrode_matrix['FZ']  = [2, 4]
electrode_matrix['F4']  = [2, 6]
electrode_matrix['F8']  = [2, 8]
electrode_matrix['FC5']  = [3, 1]
electrode_matrix['FC1']  = [3, 3]
electrode_matrix['FC2']  = [3, 5]
electrode_matrix['FC6']  = [3, 7]
electrode_matrix['T7']  = [4, 0]
electrode_matrix['C3']  = [4, 2]
electrode_matrix['CZ']  = [4, 4]
electrode_matrix['C4']  = [4, 6]
electrode_matrix['T8']  = [4, 8]
electrode_matrix['CP5']  = [5, 1]
electrode_matrix['CP1']  = [5, 3]
electrode_matrix['CP2']  = [5, 5]
electrode_matrix['CP6']  = [5, 7]
electrode_matrix['P7']  = [6, 0]
electrode_matrix['P3']  = [6, 2]
electrode_matrix['PZ']  = [6, 4]
electrode_matrix['P4']  = [6, 6]
electrode_matrix['P8']  = [6, 8]
electrode_matrix['PO3'] = [7, 3]
electrode_matrix['PO4'] = [7, 5]
electrode_matrix['O1'] = [8, 3]
electrode_matrix['OZ'] = [8, 4]
electrode_matrix['O2'] = [8, 5]

list_electrodes = ['FP1', 'AF3', 'F3', 'F7', 'FC5', 'FC1', 'C3',	'T7',	'CP5',	'CP1',	'P3',	'P7',	'PO3',	'O1',	'OZ',	'PZ',	'FP2',	'AF4', 'FZ', 'F4', 'F8', 'FC6',	'FC2',	'CZ', 'C4', 'T8', 'CP6',	'CP2',	'P4', 	'P8',	'PO4',	'O2']
data_dir = './data_preprocessed_python'
TRAIN_SIZE = 0.9
THRESHOLD = 4.5

In [ ]:
#  data_samples = []
#  labels = []
#  interval = 3
#  shift = 64
#  for sub in range(len(data)): #sub   - человек
#       for nvideo in range(NVIDEOS):
#             for ntime in range(0, (LEN_RECORD_IN_SECONDS - interval) * 128, shift):
#                    temp = data[sub][nvideo, :,  3 * NTIMES_IN_SEC + ntime : (3 + interval) *  NTIMES_IN_SEC + ntime].copy()
#                    #data_sample = np.zeros((NCANALS, NTIMES_IN_SEC + 1),  dtype=np.float16)
#                    data_sample = np.zeros((NCANALS, NTIMES_IN_SEC + 1))
#                    for i in range(NCANALS):
#                         freq, data_sample[i] = signal.welch(temp[i], fs=128.0, window='hanning', nperseg=256, noverlap=None, nfft=None, detrend='constant', return_onesided=True, scaling='density', axis=-1)
#                    data_samples.append(data_sample)
#                    labels.append(labels_bin[sub][nvideo, :])


In [ ]:
# import pickle
# with open('data_samples_psd2.pkl', 'wb') as fp:
#         pickle.dump(data_samples, fp)
#         pickle.dump(labels, fp)


In [ ]:
del data

In [ ]:
with open('data_samples_psd2.pkl', 'rb') as fp:
     data_samples =  pickle.load(fp)
     labels = pickle.load(fp)

In [ ]:
class EmotionDataset(Dataset):
    def __init__ (self, data_samples, labels, inds):  #indexes - индексы видео которые вошли в датасет, data - данные labels - метки бинарные
       self.data_samples = data_samples
       self.labels = labels
       self.data_samples = np.array(self.data_samples)[inds]
       self.labels = np.array(self.labels)[inds]
     

    def __len__(self):
       result =  len(self.data_samples)
       return result

    def __getitem__(self, item):
       sample = {}   
       sample = {"labels": self.labels[item],
                 "data": self.data_samples[item]
       }
       
       if transforms is not None:
           for t in transforms:
                sample = t(sample)
       
       return sample
                 


delta (0-3 Hz), theta (4-7
Hz), low alpha (8-9.5 Hz), high alpha (10.5-12 Hz), alpha (8-
12 Hz), low beta (13-16 Hz), mid beta (17-20 Hz), high beta
(21-29 Hz), beta (13-29 Hz), and gamma (30-50 Hz).

In [ ]:
# class Compose(object):
#     def __init__(self, transforms):
#         super(Compose, self).__init__()
#         self.transforms = transforms

#     def __call__(self, sample):
#         for t in self.transforms:
#             sample = t(sample)
#         return sample
import matplotlib 
from matplotlib import pyplot as plt
 
class to_pcc_matrix_per_freq(object):
    def __init__(self):
        super(to_pcc_matrix_per_freq, self).__init__()
  
    def __call__(self, sample: dict):
         freq_data = sample['data'].copy()
         
         freq_ranges_gc = np.array([0, 3, 4, 7, 8, 9.5, 10.5, 12, 8, 12, 13, 16, 17,20, 21, 29, 13, 29, 30, 50])
         koeff = 0.5
         freq_ranges_ind = ((freq_ranges_gc)/0.5).astype(int)
         list_canals = np.arange(NCANALS)
         freq_data_new = freq_data
         #print(freq_data_new )
         freq_data_new = scipy.stats.zscore(freq_data_new)
         #print(freq_data_new )
         pcc_matrixes = []
         for s in range(10):
            ind_begin = freq_ranges_ind[s * 2]
            ind_end = freq_ranges_ind[s * 2 + 1]
            pcc_matrix = np.zeros((27, 27))
            for i in range(NCANALS):
                pcc_matrix[electrode_matrix[list_electrodes[i]][0] * 3, electrode_matrix[list_electrodes[i]][1] * 3]  = (freq_data_new[i, ind_begin : ind_end ]).sum()#/sum_canals[i]
            pcc_matrixes.append(pcc_matrix)
            #print(pcc_matrix)
         sample['data'] = np.array(pcc_matrixes)
         #print(sample['data'])
         return(sample)




# class to_pcc_matrix(object):
#     def __init__(self):
#         super(to_pcc_matrix, self).__init__()
        

#     def __call__(self, sample: dict):
#         canal_data = sample['data'].copy()
#         list_canals = np.arange(NCANALS)
#         random.shuffle(list_canals)
#         canal_data_new = [canal_data[list_canals[i]] for i in range(32)]
#         pcc_matrix = np.cov(canal_data_new)
#         for i in range(NCANALS):
#            for j in range(i + 1):
#               pcc_matrix[i, j] = pcc_matrix[i, j]/(np.std(canal_data_new[i]) * np.std(canal_data_new[j]))
#               pcc_matrix[j, i] = pcc_matrix[i, j]
#         sample['data'] = pcc_matrix
#         #print(pcc_matrix)
#         return(sample)

class ToTensor(object):

    def __init__(self, ):
        super(ToTensor, self).__init__()

    def __call__(self, sample: dict):
        return {"labels": torch.tensor(sample["labels"], dtype=torch.long),
                "data": torch.tensor(sample["data"], dtype=torch.float32),
                } 


transforms = [to_pcc_matrix_per_freq(),
                   ToTensor()]                       

In [ ]:
def get_conv_module(n_input_channels,n_output_channels):
  conv = nn.Conv2d(n_input_channels, n_output_channels, kernel_size = 3, stride=1, padding='same')
  maxpool = nn.MaxPool2d(kernel_size = 2)
  bn = nn.BatchNorm2d(n_output_channels)
  relu = nn.ReLU()
  result = torch.nn.Sequential(conv, maxpool, bn, relu)
  return(result)

In [ ]:
class EmotionNet(torch.nn.Module): 
   def __init__(self, hcanals, wcanals, tures, ntimes_in_sample):
      super().__init__()
      n_input_channels = [10, 32, 64, 128]#, 256]
      n_output_channels = [32, 64, 128, 256]#, 512]
      self.convs = nn.ModuleList([get_conv_module(n_input_channels[i], n_output_channels[i]) for i in range(3)])
      self.conv = nn.Conv2d(n_input_channels[3], n_output_channels[3], kernel_size = 3, stride=1, padding='same')
      self.flat = nn.Flatten(1, 3)
      self.fc = nn.Linear(2304, 2)

      
   def forward(self, input):
      #print(f"input_shape = {input.shape}")
      #input = input.unsqueeze(3)
      #input = input.permute(0, 3, 1, 2)
      #input (bs, h=32, w=32, 1)

      output = input
      for i, conv in enumerate(self.convs):
            output = conv(output)
            #print(outputs[i])
      output = self.conv(output)
      output = self.flat(output)
      #output (bs, s * 256,  h=9, w=9)
      output = self.fc(output)
      return output



In [ ]:
writer = None

In [ ]:
def train(model, loader, criterion, optimizer, device, val_dataloader, val_f1_min,  description, type_emotion, batch = None, writer = None):
    model.train()
    train_loss = []
    inputs = []
    torch.autograd.set_detect_anomaly(True)
   
    #lr_scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(optimizer)#, mode='min', factor=0.1, patience=10, threshold=0.0001, threshold_mode='rel', cooldown=0, min_lr=0, eps=1e-08, verbose=False)
    for i , batch in enumerate(loader):#, total=len(loader), desc="training...", position=0 , leave = True)):
            model.train()
            optimizer.zero_grad()
            src  = batch['data'].to(device)
            trg = batch['labels'][:, type_emotion]
            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            train_loss.append(loss.item())
            loss.backward()
            optimizer.step()
            if writer:
                writer.add_scalar(f'{description}/training loss per batch',
                                  loss.item(),
                                  i)

            # if (i % 100 == 0):
            #     acc, f1 = (calculate_predictions(model, val_dataloader))
            #     if (f1 > val_f1_min):
            #           val_f1_min      = f1
            #           torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val.tgz"))
            #break
    return np.mean(train_loss), val_f1_min

def evaluate(model, loader, criterion, device, writer, description, type_emotion):
    
    model.eval()
    epoch_loss = 0
    history = []
  
    with torch.no_grad():
    
        for s, batch in enumerate(loader):#, total=len(loader), desc="validating...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]



            levels_pred = model(src)  # B x (2 * NUM_PTS)
            #print(levels_pred.shape)
            levels_pred = levels_pred.cpu()
            loss = criterion(levels_pred, trg) 
            epoch_loss += loss.item() 


            if writer:
                writer.add_scalar(f'{description}/val loss per batch',
                                  loss.item(),
                                  s)
        
    return epoch_loss / s
from sklearn.metrics import accuracy_score, confusion_matrix,classification_report

def calculate_predictions(model, loader, type_emotion, show):
    model.eval()
    epoch_loss = 0
    history = []
    real = []
    pred = []
    with torch.no_grad():

        for i, batch in enumerate(loader):#, total=len(loader), desc="predicting...", position=0 , leave = True)):
            src  = batch['data'].to(device)
            #print(src.shape)
            trg = batch['labels'][:, type_emotion]
         

            levels_pred = model(src)  # B x (2 * NUM_PTS)
            levels_pred = levels_pred.cpu()
            #print(levels_pred.shape)
            trg_pred = levels_pred.argmax(1)
            
            real.extend(trg)
            pred.extend(trg_pred) 

        if show:    
            print(accuracy_score(real, pred)) 
            print(confusion_matrix(real, pred))  
            print(classification_report(real, pred))  
        f1 = ((f1_score(real, pred, average = 'binary', pos_label = 0))  + (f1_score(real, pred, average = 'binary', pos_label = 1)))/2
        return (accuracy_score(real, pred)) , f1
        #plt.hist(real)
def train_loop(description, type_emotion, n_epochs = 10):
    #args.epochs = 10
    #criterion =  fnn.mse_loss
    train_loss_min = 10000
    val_f1_min = -10000

    #batch = next(iter(train_dataloader))
    for epoch in range(n_epochs):
          #logger.info(f"Starting epoch {epoch + 1}/{args.epochs}.")
    
          train_loss, val_f1_min  = train(model, train_dataloader, criterion, optimizer ,device, val_dataloader, val_f1_min ,  description , type_emotion, None,  writer)
          #if epoch % 500 == 0:
          if writer:
                writer.add_scalar(f"{description}/training loss per epoch",
                                        train_loss,
                                        epoch)
          #print(train_loss)

          if (train_loss < train_loss_min):
                 train_loss_min      = train_loss
                 torch.save({
                         'model_state_dict': model.state_dict(),
                         'optimizer_state_dict': optimizer.state_dict(),
                       },
                       os.path.join(args.output_dir, "train.tgz")
            )  

          #val_loss = evaluate(model, val_dataloader, criterion, device,  writer, description )
          # #break
          #print(val_loss)
          #if writer:
          #      writer.add_scalar(f"{description}/val loss per epoch",
          #                        val_loss,
          #                        epoch)
          acc, f1 = (calculate_predictions(model, val_dataloader, type_emotion, False))
          print(acc, f1)
          if (acc > val_f1_min):
                       val_f1_min      = acc
                       torch.save({'model_state_dict': model.state_dict(),    'optimizer_state_dict': optimizer.state_dict(),}, os.path.join(args.output_dir, f"val_{description}.tgz")) 

In [ ]:
def get_model():
  model = EmotionNet(HCANALS, WCANALS, NFEATURES, NTIMES_IN_SAMPLE).to(device)
  return model

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model = get_model()
def initialize_weights(m):
    if hasattr(m, 'weight') and m.weight.dim() > 1:
        nn.init.xavier_uniform_(m.weight.data)

model.apply(initialize_weights)

EmotionNet(
  (convs): ModuleList(
    (0): Sequential(
      (0): Conv2d(10, 32, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (1): Sequential(
      (0): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
    (2): Sequential(
      (0): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=same)
      (1): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
      (2): BatchNorm2d(128, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (3): ReLU()
    )
  )
  (conv): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), paddin

#Разбиение с ликом

In [ ]:
type_emotion  = 0

In [ ]:
sample = ( train_dataset.__getitem__(20) )
print(sample['data'][0])


In [ ]:
from sklearn.model_selection import StratifiedKFold , KFold
from sklearn.metrics import f1_score, accuracy_score
interval = 3
shift = 1
koeff = 0.8
import pandas as pd    
type_emotion  = 0
# #inds = np.arange(2280)
#inds = np.arange(72960)
args.batch_size = 256

# #inds = np.arange(12800)
# random.shuffle(inds)
acc_all = []
f1_all = []
inds = np.arange(len(data_samples))
kf = KFold(n_splits=5, shuffle = True)
for fold,  (inds_train, inds_test) in enumerate(kf.split(inds)):
    print(f"fold = {fold}")
    #if fold == 0:
      #continue       
    #transforms_random = RandomAugmentation([add_noise, reset_part_in_freq, reset_part_in_time, None], 0.2)
    #transforms = [RandomAugmentation([add_noise(), reset_part_in_freq(0.2), reset_part_in_time(0.2), None], 0.2), to_head_matrix(),ToTensor()]   
    transforms = [to_pcc_matrix_per_freq(),   ToTensor()]       
    #transforms = [RandomAugmentation([add_noise(), None], 0.5), to_head_matrix(),ToTensor()]   
    #transforms = [RandomAugmentation([reset_part_in_time(0.4), None], 0.1), to_head_matrix(),ToTensor()]   

    train_dataset = EmotionDataset(data_samples, labels, inds_train)
    train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
                            pin_memory=True, shuffle=True, drop_last=True)


    val_dataset = EmotionDataset(data_samples, labels, inds_test)
    val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
                              pin_memory=True, shuffle=False, drop_last=False)
    #break
    device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
    model = get_model()
    model.apply(initialize_weights)
    criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
    #optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
    optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
    description = f'psd_model_{fold}'
    train_loop(description, 0, 20)
    model_state  = torch.load(os.path.join(args.output_dir, f"val_{description}.tgz"))
    model.load_state_dict(model_state['model_state_dict'])
    acc, f1 = calculate_predictions(model, val_dataloader, type_emotion, True)
    print(f"************f1 = {f1} acc = {acc}********************")
    acc_all.append(acc)
    f1_all.append(f1)
        
    pd.DataFrame(f1_all).to_csv("f1_val_psd_leak_result.csv")
    pd.DataFrame(acc_all).to_csv("acc_val_psd_leak_result.csv")






  

fold = 0
0.6955523574561403 0.6293091481796738
0.7086074561403509 0.6721159500677063
0.7248834978070176 0.681517806229814
0.7430098684210527 0.712450113944007
0.7426329495614035 0.7121218620288612
0.7503769188596491 0.7273633557116426
0.7508223684210527 0.7316958632044008
0.7664473684210527 0.738987060371418
0.7660361842105263 0.7265826211255043
0.7693256578947368 0.7393015525771831
0.7747053179824561 0.7490882745480334
0.7625068530701754 0.7498726819438277
0.7760759320175439 0.7454214649937395
0.784094024122807 0.759238352338075
0.7894394188596491 0.7657938854503679
0.780187774122807 0.7632359275178187
0.7878974780701754 0.7650762975197533
0.7912554824561403 0.7714553670632502
0.7835800438596491 0.7693537025665274
0.7927288925438597 0.7731835912097231
0.7927288925438597
[[ 7284  3500]
 [ 2549 15851]]
              precision    recall  f1-score   support

           0       0.74      0.68      0.71     10784
           1       0.82      0.86      0.84     18400

    accuracy           

In [ ]:
# with open('train_dataset.pkl', 'wb') as fp:
#        pickle.dump(train_dataset, fp)

In [ ]:
# with open('val_dataset.pkl', 'wb') as fp:
#        pickle.dump(val_dataset, fp)

In [ ]:
# with open('train_dataset.pkl', 'rb') as fp:
#        train_dataset = pickle.load(fp)
# with open('val_dataset.pkl', 'rb') as fp:
#         val_dataset = pickle.load(fp)


# train_dataloader = DataLoader(train_dataset, batch_size=args.batch_size, num_workers=1,
#                                 pin_memory=True, shuffle=True, drop_last=True)


# val_dataset = EmotionDataset(data, labels_bin, interval, shift, inds, 'test')
#val_dataloader = DataLoader(val_dataset, batch_size=args.batch_size, num_workers=1,
#                                pin_memory=True, shuffle=False, drop_last=False)        
        

In [ ]:
criterion = nn.CrossEntropyLoss(reduction = 'mean')#torch.nn.MSELoss()
#optimizer = optim.SGD(model.parameters(), lr=3e-5, momentum = 0.9)#, weight_decay=args.weight_decay)
optimizer = optim.Adam(model.parameters(), lr=3e-4)#, momentum = 0.9)#, weight_decay=args.weight_decay)
# print(train_dataset.cnt)
# print(val_dataset.cnt)
# print(files[ind_train])
# print(files[ind_val])

In [ ]:
val_loss = evaluate(model, val_dataloader, criterion, device)
# #break
print(val_loss)

In [ ]:
model_state  = torch.load(os.path.join("/content/drive/MyDrive/MADE/Project/CNN_models/", f"val_5.tgz"))
        #   #model = Seq2Seq(enc, dec, SRC_PAD_IDX, TRG_PAD_IDX, device)
model.load_state_dict(model_state['model_state_dict'])
print(calculate_predictions(model, val_dataloader))

predicting...:   0%|          | 0/57 [00:00<?, ?it/s]

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7f19eb5c80e0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1328, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py", line 1320, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.7/multiprocessing/process.py", line 151, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process


0.7451343201754386
[[3232 2156]
 [1563 7641]]
              precision    recall  f1-score   support

           0       0.67      0.60      0.63      5388
           1       0.78      0.83      0.80      9204

    accuracy                           0.75     14592
   macro avg       0.73      0.72      0.72     14592
weighted avg       0.74      0.75      0.74     14592

(0.7451343201754386, 0.7195284609627088)
